In [1]:
import numpy as np
import pandas as pd
import json
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import pickle as pkl
import imblearn
np.random.seed(1332)
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
#import seaborn as sns
import statsmodels.api as sm
%matplotlib inline
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import sys                                               #### Filling the missing values using KNN because the dataset is very less
from impyute.imputation.cs import fast_knn
import imblearn                                      ### sampling the data using the smote method
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
import sklearn
import xgboost
from xgboost import XGBClassifier              ### XGBoost classification method
import sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from xgboost import cv

In [2]:
xls = pd.ExcelFile('Coronna Data CERTAIN with KVB edits.xlsx')
sheet2 = pd.read_excel(xls, 'BL+3M')
data1 = sheet2.copy()

In [3]:
data1 = data1.drop(columns = ['SubjectID', 'CDate', 'Match'])
data1 = data1.drop(columns = ['SubjectID.1', 'UNMC_id.1', 'CDate.1', 'init_group.1', 'grp.1', 'UNMC_id.2', 'grp.2', 'init_group.2', 'CDate.1', 'futime.1'])
data1 = data1.drop(columns = ['init_group', 'futime'])
#Drop the features with more than 70% NA values (other than smkyrs)
data1 = data1.drop(columns = ['statin_use', 'rfstatus_impute', 'ccpstatus_impute', 'statin_use.1'])
#Drop the features that are considered not usable
data1 = data1.drop(columns = ['nonpresNSAIDs_use', 'NSAIDs_use','nonpresNSAIDs_use.1', 'NSAIDs_use.1'])
#Distinguish different features with the same name
data1 = data1.rename(columns={"usresultsIgA.1": "usresultsIgA_BL", "usresultsIgG.1": "usresultsIgG_BL", "usresultsIgM.1": "usresultsIgM_BL", 'seatedbp1.1': 'seatedbp1_BL',
                              'seatedbp2.1': 'seatedbp2_BL', 'pres_mtx.1': 'pres_mtx_BL', 'pres_arava.1': 'pres_arava_BL', 'pres_azulfidine.1': 'pres_azulfidine_BL', 
                              'pres_plaquenil.1': 'pres_plaquenil_BL', 'pres_imuran.1': 'pres_imuran_BL', 'pres_minocin.1': 'pres_minocin_BL', 'pres_pred.1': 'pres_pred_BL',
                              'statin_use.1': 'statin_use_BL', 'tender_jts_28.1': 'tender_jts_28_BL', 'BLswollen_jts_28': 'swollen_jts_28_BL',
                              'BLmd_global_assess': 'md_global_assess_BL', 'BLpt_global_assess': 'pt_global_assess_BL', 'BLdi': 'di_BL', 'BLpt_pain': 'pt_pain_BL', 'BLusresultsCRP': 'usresultsCRP_BL',
                              'DAS28-CRP BL': 'DAS28-CRP_BL', })
data1['BMI'] = data1['weight'] / (data1['height'] ** 2) * 703 
data1['IFNβ activity final'][234] = None
data1['IFNα activity final'][234] = None
data1['IFNβ/α ratio final'][234] = None

/state/partition1/job-15753178/ipykernel_1997006/3999710629.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['IFNβ activity final'][234] = None
/state/partition1/job-15753178/ipykernel_1997006/3999710629.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['IFNα activity final'][234] = None
/state/partition1/job-15753178/ipykernel_1997006/3999710629.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['IFNβ/α rati

In [28]:
data=data1
data.shape

(296, 81)

In [29]:
#data = data[data['3MResponse'] != 'Unknown']
import math
def flag_na(row):
    if math.isnan(row['IFNβ/α ratio final']):
        return 1
    else:
        return 0
#This line of code iterate over each row of IFN and add a new feature to indicate whether IFN is missing
data['IFN Missing'] = data.apply(lambda row: flag_na(row), axis = 1)
data = data.fillna({'Type I IFN activity':0, 'IFNβ activity final':0, 'IFNα activity final':0, 'IFNβ/α ratio final':0})
data = data.drop(columns = ['seatedbp1', 'seatedbp2', 'pres_mtx', 'pres_arava', 'pres_azulfidine', 'pres_plaquenil',
                           'pres_pred', 'md_global_assess', 'pt_global_assess', 'di', 'pt_pain', 'usresultsIgA', 
                           'usresultsIgG', 'usresultsIgM'])

#pres_imuran, pres_minocin, pres_minocin_BL, num_tnf, num_nontnf are all zeros
#This kind of variables will not provide useful information
data = data.drop(columns = ['pres_imuran', 'pres_minocin', 'pres_minocin_BL', 'num_tnf', 'num_nontnf'])
#these are the columns that have correlation with the target feature
#We will need to delete these values to avoid redundent information and noise
data = data.drop(columns = ['ara_func_class', 'tender_jts_28', 'swollen_jts_28', 'usresultsCRP'])
data = data[data['drinker'].notnull()]
#Because it does not have 0 value for normal data, thus here zero will indicate it has a missing value rather than a actual number
data = data[data['usresultsRF'].notnull()]
#Because it does not have 0 value for normal data, thus here zero will indicate it has a missing value rather than actual number
data['usresultsCCP3'] = data['usresultsCCP3'].replace(np.nan, 0)
#Drops the rows with missing value for seatedbp1_BL
data = data[data['seatedbp1_BL'].notnull()]
data = data[data['md_global_assess_BL'].notnull()]
data = data.fillna({'usresultsIgA_BL': 0, 'usresultsIgG_BL':0, 'usresultsIgM_BL':0})
data = data[data['BMI'].notnull()]
data = data.drop(columns = ['UNMC_id'])
data = data[data['final_education'].notnull()]
data = data[data['final_education'] != "don't remember"]
data = data.drop(columns = ['smkyrs', 'numcigs'])

In [30]:
from sklearn import preprocessing
encoders = dict()

for col_name in ['grp', 'gender', 'final_education', 'race_grp', 'newsmoker', 'drinker']:
    series = data[col_name]
    label_encoder = preprocessing.LabelEncoder()
    data[col_name] = pd.Series(
        label_encoder.fit_transform(series[series.notnull()]),
        index=series[series.notnull()].index
    )
    encoders[col_name] = label_encoder
data3 = data.copy()
data3 = data3[data3['ethnicity'].notnull()]
data3 = data3[data3['newsmoker'].notnull()]

In [31]:
data3 = data3.drop(columns = ['DAS28-CRP 3m', 'DAS28-CRP_BL'])
data3 = data3.drop(columns = ['weight', 'height','hxunstab_ang'])

In [32]:
data3['3MResponse'].unique()

array(['Good', 'No Response', 'Moderate', 'Unknown'], dtype=object)

/state/partition1/job-15753178/ipykernel_1982559/2061900728.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop('UNMC_id',1)


In [33]:
final_df_1 = data3[data3['3MResponse'] != 'Unknown']### We don't need unknown category in our classification data so removing that category
new_df = data3[data3['3MResponse'] == 'Unknown']
model_label = LabelEncoder()
final_df_1['3MResponse'] = model_label.fit_transform(final_df_1['3MResponse'].astype('str'))

/state/partition1/job-15753178/ipykernel_1997006/1092632713.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df_1['3MResponse'] = model_label.fit_transform(final_df_1['3MResponse'].astype('str'))


In [34]:
# def KNN(new_df):
#     sys.setrecursionlimit(100000) 
#     imputed_training=fast_knn(new_df.values, k=15)
#     return imputed_training
# imputed_training = KNN(final_df_1)
# c = list(final_df_1.columns)
# df_without_nulls = pd.DataFrame(imputed_training, columns=c)
df_without_nulls = final_df_1

In [441]:
rs = 901
X = df_without_nulls.drop('3MResponse',1)                            ### Dividing the dataframe into X and y set
y = df_without_nulls['3MResponse']
X_train_norm, X_val_norm, Y_train, Y_val = train_test_split(X, y, test_size=0.1,random_state = rs) 

/state/partition1/job-15753178/ipykernel_1997006/3078864793.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X = df_without_nulls.drop('3MResponse',1)                            ### Dividing the dataframe into X and y set


In [442]:
new_df['3MResponse'] = new_df['3MResponse'].replace('Unknown', np.nan)

/state/partition1/job-15753178/ipykernel_1997006/3023486207.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['3MResponse'] = new_df['3MResponse'].replace('Unknown', np.nan)


In [443]:
X_train_norm['3MResponse'] = Y_train

In [444]:
combined_train_df = pd.concat([X_train_norm, new_df])

In [445]:
def KNN(new_df):
    sys.setrecursionlimit(100000) 
    imputed_training=fast_knn(new_df.values, k=15)
    return imputed_training
imputed_training = KNN(combined_train_df)
c = list(combined_train_df.columns)
unknown_combined = pd.DataFrame(imputed_training, columns=c)
#unknown_combined = combined_train_df

In [446]:
for i in range(len(unknown_combined['3MResponse'])):
    if unknown_combined['3MResponse'][i] <= 0.5: 
        unknown_combined['3MResponse'][i] = 0
    elif unknown_combined['3MResponse'][i] >0.5 and unknown_combined['3MResponse'][i]<=1.5:
        unknown_combined['3MResponse'][i] = 1
    else:
        unknown_combined['3MResponse'][i] = 2

In [447]:
X_unknown = unknown_combined.drop('3MResponse',1)                            ### Dividing the dataframe into X and y set
y_unknown = unknown_combined['3MResponse']

/state/partition1/job-15753178/ipykernel_1997006/2470334778.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X_unknown = unknown_combined.drop('3MResponse',1)                            ### Dividing the dataframe into X and y set


In [448]:
labels = ['grp','gender','final_education','race_grp','newsmoker','drinker','ara_func_class']     ### normalizing the dataset

for i in X_unknown.columns:
    if i not in labels:
        mean = X_unknown[i].mean()
        std = X_unknown[i].std()
        X_unknown[i] = (X_unknown[i] - mean) / std
        X_val_norm[i] = (X_val_norm[i] - mean) / std

In [449]:
labels = ['ethnicity','hxstroke','pres_imuran_BL']    #### This columns has same value for every row so when I divide by mean and std the value goes to infinity
for i in labels:
    X_unknown = X_unknown.drop(i,1)
    X_val_norm = X_val_norm.drop(i,1)

/state/partition1/job-15753178/ipykernel_1997006/856867150.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X_unknown = X_unknown.drop(i,1)
/state/partition1/job-15753178/ipykernel_1997006/856867150.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X_val_norm = X_val_norm.drop(i,1)


In [450]:
X_unknown['y'] = y_unknown
X_val_norm['y'] = Y_val

In [451]:
X_train_norm = X_unknown.drop(columns=['y'])
Y_train = X_unknown['y']
X_val_norm_1 = X_val_norm.drop(columns=['y'])
Y_val = X_val_norm['y']

In [452]:
def sampling(over_under,X,y):
    if over_under == 'under':
        sampler = imblearn.under_sampling.RandomUnderSampler()
        X_under, y_under = sampler.fit_resample(X, y)
        return X_under,y_under
    elif over_under == 'over':
        ros = RandomOverSampler(random_state=rs)
        X_over, y_over = ros.fit_resample(X, y)
        return X_over,y_over
    else:
        sampler = SMOTE()
        X_smote, y_smote = sampler.fit_resample(X, y)
        return X_smote,y_smote
X_final,y_final = sampling('smote',X_train_norm,Y_train)

In [453]:
from mlxtend.classifier import StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.neural_network import MLPClassifier
lr = sklearn.linear_model.LogisticRegression(multi_class='multinomial', solver='saga', penalty = 'elasticnet', C = 1, l1_ratio = 0.5)  # defining meta-classifier
SVM = svm.SVC(kernel = 'linear')
xgb = XGBClassifier(n_estimators = 200, subsample=0.8, max_depth=50, eta=0.5, gamma=1, min_child_weight=10, alpha=1)   
rf = sklearn.ensemble.RandomForestClassifier(n_estimators=474, oob_score = True, criterion='gini',max_depth=90, min_samples_split=10, max_features='auto', min_samples_leaf = 6)
mlp = MLPClassifier(activation = "relu", alpha = 0.1, hidden_layer_sizes = (10,10,10), learning_rate = "constant", max_iter = 2000, random_state = 1000)
NB = GaussianNB() 
knn = KNeighborsClassifier(n_neighbors=10)
clf_stack = StackingClassifier(classifiers =[rf, xgb], meta_classifier = lr,use_probas=True)
clf_stack = clf_stack.fit(X_final, y_final)
trans = clf_stack.predict_meta_features(X_final)
print("Outputs", trans, trans.shape)
kfold = StratifiedKFold(n_splits=10)
results = cross_val_score(clf_stack, X_final, y_final, cv=10)
print("sklearn:::", results.mean())
print(f"Training accuracy is {clf_stack.score(X_final, y_final)}")
print(f"Testing accuracy is {clf_stack.score(X_val_norm_1, Y_val)}")

/ext3/miniconda3/lib/python3.9/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/ext3/miniconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:35:28] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Outputs [[0.18410288 0.23892751 0.57696961 0.03162954 0.2964817  0.67188877]
 [0.34794042 0.42861483 0.22344475 0.23990263 0.71372503 0.04637237]
 [0.21011257 0.58872392 0.20116351 0.08120476 0.78275597 0.13603926]
 ...
 [0.17029093 0.16139294 0.66831613 0.10298061 0.04171484 0.85530454]
 [0.13622818 0.16605672 0.6977151  0.04112473 0.06490113 0.89397413]
 [0.15472282 0.17103503 0.67424215 0.04815023 0.04047134 0.91137844]] (285, 6)


/ext3/miniconda3/lib/python3.9/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/ext3/miniconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:35:29] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/ext3/miniconda3/lib/python3.9/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/ext3/miniconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[12:35:38] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/ext3/miniconda3/lib/python3.9/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/ext3/miniconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:35:39] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/ext3/miniconda3/lib/python3.9/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/ext3/miniconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:35:40] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
sklearn::: 0.5211822660098522
Training accuracy is 0.9473684210526315
Testing accuracy is 0.2692307692307692


In [359]:
model4 = None
def xgboost(X_train_norm, X_test,Y_train, Y_test):
    global model4
    model4 = XGBClassifier(n_estimators = 200, subsample=0.8, max_depth=50, eta=0.5, gamma=1, min_child_weight=10, alpha=1)

    model4.fit(X_train_norm, Y_train)
    kfold = StratifiedKFold(n_splits=10)
    results = cross_val_score(model4, X_train_norm, Y_train, cv=10)
    print("sklearn:::", results, results.mean())
    print(f"Training accuracy is {model4.score(X_train_norm, Y_train)}")
    print(f"Testing accuracy is {model4.score(X_test, Y_test)}")
xgboost(X_final, X_val_norm_1, y_final, Y_val)

/ext3/miniconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:21:12] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/ext3/miniconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:21:12] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/ext3/miniconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[12:21:16] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/ext3/miniconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:21:17] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/ext3/miniconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:21:17] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
sklearn::: [0.44827586 0.37931034 0.44827586 0.34482759 0.44827586 0.39285714
 0.67857143 0.5        0.64285714 0.67857143] 0.4961822660098522
Training accuracy is 0.9614035087719298
Testing accuracy is 0.19230769230769232


In [267]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
model5 = None
def logistic_regress(X_train_norm, X_test, Y_train,Y_test):
    global model5
    model5 = sklearn.linear_model.LogisticRegression(multi_class='multinomial', solver='saga', penalty = 'elasticnet', C = 1, l1_ratio = 0.5, random_state = rs)
    model5.fit(X_train_norm, Y_train)
    kfold = StratifiedKFold(n_splits=10)
    results = cross_val_score(model5, X_train_norm, Y_train, cv=10)
    print("sklearn:::", results.mean())
    print(f"Training accuracy is {model5.score(X_train_norm, Y_train)}")
    print(f"Testing accuracy is {model5.score(X_test, Y_test)}")
logistic_regress(X_final, X_val_norm_1, y_final, Y_val)

/ext3/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/ext3/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/ext3/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/ext3/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/ext3/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/ext3/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: T

sklearn::: 0.4002463054187192
Training accuracy is 0.6140350877192983
Testing accuracy is 0.5


/ext3/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/ext3/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/ext3/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/ext3/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [176]:
model3 = None
def random_forest(X_train_norm, X_test, Y_train,Y_test):
    global model3
    model3 = sklearn.ensemble.RandomForestClassifier(n_estimators=474, oob_score = True, criterion='gini',max_depth=90, min_samples_split=10, max_features='auto', min_samples_leaf = 6)
    model3.fit(X_train_norm, Y_train)
    print(model3.oob_score_)
    print(f"Training accuracy is {model3.score(X_train_norm, Y_train)}")
    print(f"Testing accuracy is {model3.score(X_test, Y_test)}")
random_forest(X_final, X_val_norm_1, y_final, Y_val)

/ext3/miniconda3/lib/python3.9/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


0.5520833333333334
Training accuracy is 0.9270833333333334
Testing accuracy is 0.5769230769230769


In [36]:
from sklearn.ensemble import StackingClassifier
clf_stack_1 = None
clf_svc = sklearn.svm.SVC(C = 50, degree = 1, gamma = "auto", kernel = "rbf", probability = True)

# Initializing Multi-layer perceptron  classifier
clf_mlp = MLPClassifier(activation = "relu", alpha = 0.1, hidden_layer_sizes = (10,10,10),
                            learning_rate = "constant", max_iter = 2000, random_state = 1000)

# Initialing Nu Support Vector classifier
clf_nusvc = sklearn.svm.NuSVC(degree = 1, kernel = "rbf", nu = 0.25, probability = True)

# Initializing Random Forest classifier
clf_rfc = sklearn.ensemble.RandomForestClassifier(n_estimators = 500, criterion = "gini", max_depth = 10,
                                     max_features = "auto", min_samples_leaf = 0.005,
                                     min_samples_split = 0.005, n_jobs = -1, random_state = 1000)
lr = sklearn.linear_model.LogisticRegression(multi_class='multinomial', solver='saga', penalty = 'elasticnet', C = 1, l1_ratio = 0.5)  # defining meta-classifier


xgb = XGBClassifier(n_estimators = 200, subsample=0.8, max_depth=50, eta=0.5, gamma=1, min_child_weight=10, alpha=1)   
rf = sklearn.ensemble.RandomForestClassifier(n_estimators=474, oob_score = True, criterion='gini',max_depth=90, min_samples_split=10, max_features='auto', min_samples_leaf = 6)

clf_stack_1 = StackingClassifier(estimators =[('xgb',XGBClassifier(n_estimators = 200, subsample=0.8, max_depth=50, eta=0.5, gamma=1, min_child_weight=10, alpha=1)),('rf',sklearn.ensemble.RandomForestClassifier(n_estimators=474, oob_score = True, criterion='gini',max_depth=90, min_samples_split=10, max_features='auto', min_samples_leaf = 6))], final_estimator = lr)
clf_stack = clf_stack_1.fit(X_final, y_final)
trans = clf_stack_1.fit_transform(X_final, y_final)
print("transss", trans,trans.shape)
kfold = StratifiedKFold(n_splits=10)
results = cross_val_score(clf_stack, X_final, y_final, cv=10)
print("sklearn:::", results.mean())
print(f"Training accuracy is {clf_stack.score(X_final, y_final)}")
print(f"Testing accuracy is {clf_stack.score(X_val_norm_1, Y_val)}")

/ext3/miniconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:21:13] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/ext3/miniconda3/lib/python3.9/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/ext3/miniconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:21:14] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/ext3/miniconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[10:22:38] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/ext3/miniconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:22:38] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/ext3/miniconda3/lib/python3.9/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/ext3/miniconda3/lib/python3.9/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/ext3/miniconda3/lib/python3.9/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/ext3/miniconda3/lib/python3.9/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/ext3/miniconda3/lib/python3.9/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


sklearn::: 0.765625
Training accuracy is 0.9755351681957186
Testing accuracy is 0.6071428571428571
